# Overview
**Overview**

This Jupyter Notebook establishes a foundational Retrieval-Augmented Generator (RAG) pipeline, replicating the core components and structure of the Cohere toolkit. The baseline pipeline serves as a benchmark for evaluating the effectiveness of various RAG improvement techniques.

**Components of the RAG Pipeline**

The RAG pipeline consists of the following components:

1. **Vector Store and Retriever**: The pipeline uses a MongoDB-based vector store and a retriever based on the `langchain_mongodb` library. The vector store is initialized with an OpenAI embedding model, and the retriever is configured to retrieve up to 20 documents based on similarity.
2. **LLM Model**: The pipeline uses a ChatOpenAI model as the Language Model (LLM) component.
3. **Prompt Template**: A custom prompt template is defined to provide context to the LLM model. The template includes a safety preamble, basic rules, task context, style guide, and instructions.
4. **RAG Function**: The `call_openai` function is defined to process user inputs, retrieve relevant documents, and generate answers using the LLM model.

**Testing the RAG Pipeline**

The pipeline is tested using a sample question, and the output is printed to the console. Additionally, the pipeline is evaluated using a test dataset (`test_dataset_it.json`) containing questions from the IT department.

**Evaluation Metrics**

The pipeline is evaluated using the following metrics:

1. **Answer Relevancy**: Measures the relevance of the generated answer to the input question.
2. **Faithfulness**: Measures the accuracy of the generated answer based on the retrieved documents.
3. **Context Recall**: Measures the proportion of relevant documents retrieved by the pipeline.
4. **Context Precision**: Measures the proportion of retrieved documents that are relevant to the input question.

**Results**

The evaluation results are stored in a pandas DataFrame and written to a CSV file (`eval_result_test_dataset_it_baseline.csv`). The results provide a baseline for evaluating the effectiveness of various RAG improvement techniques.

**Code Structure**

The code is organized into the following sections:

1. **Importing Libraries**: The necessary libraries are imported, including `langchain_core`, `langchain_mongodb`, `langchain_openai`, `pymongo`, and `ragas`.
2. **Defining the RAG Pipeline**: The vector store and retriever, LLM model, prompt template, and RAG function are defined.
3. **Testing the RAG Pipeline**: The pipeline is tested using a sample question and a test dataset.
4. **Evaluating the RAG Pipeline**: The pipeline is evaluated using the `ragas` library and the evaluation results are stored in a pandas DataFrame.

Overall, this notebook provides a comprehensive example of a RAG pipeline and its evaluation using various metrics.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
# Initialize OpenAI llm and embedding model

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database

client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")
collection_name="enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

In [ ]:
# Initialize vector store and retriever

vector_store = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = atlas_collection,
    index_name = index_name
)

retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 20}  # default "score_threshold": 0.75 
)

## RAG function

In [8]:
# Define a prompt template
import pprint
def call_openai(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"),max_tokens=200)

   # Remove duplicate retrieved documents
   def remove_rep(docs):
      docs_content = []
      unique_docs = []
      for doc in docs:
            if doc.page_content not in docs_content:
                  docs_content.append(doc.page_content)
                  unique_docs.append(doc)
      return unique_docs
   
   def format_docs(docs):
      contexts = remove_rep(docs)
      return "\n\n".join(doc.page_content for doc in contexts)


   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   answer = rag_chain.invoke(question)
   retrieved_docs = remove_rep(retriever.invoke(question))

   return{
       'answer': answer,
       'contexts': retrieved_docs
      }

# RAG pipeline evaluation

**Load test questions dataset of IT Department.**

## Test data set prep

In [53]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [56]:
def extract_page_content(documents):
    """
    Reads a list of Document objects and return a list of str of Document objects' page_content.

    Args:
        A list of Document objects

    Returns:
        A list of strings, which are the page_content of the Document objects
    """
    return [doc.page_content for doc in documents]

In [54]:
# Example usage:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test-dataset/",file_name)
data_to_test = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [58]:
# Generate all the answers for the questions in the dataset
# Store the answers and retrieved documents as contexts to lists
answers = []
contexts = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    contexts.append(answer['contexts'])
    answers.append(answer['answer'])

In [59]:
# update the dataset with answers and contexts
data_to_test['answers'] = answers
data_to_test['contexts'] = contexts

In [ ]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts'] = data_to_test['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [60]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answers'])
contexts = list(data_to_test['contexts'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_baseline = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_baseline = Dataset.from_dict(data_baseline)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "Cohere_RAG_Eval"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [62]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset_baseline,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [63]:
result

{'answer_relevancy': 0.9602, 'faithfulness': 0.5685, 'context_recall': 0.5375, 'context_precision': 0.6542}

In [64]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The role of the Senior Director responsible fo...,[the Senior Director responsible for Analytics...,The role of the Senior Director responsible fo...,0.921495,1.000000,0.272727,0.8875
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[to identify strengths and areas for improveme...,Identifying and addressing growth areas in sel...,0.990183,0.416667,0.333333,0.0000
2,What forms of unethical behavior are strictly ...,"In the recruitment process of Inc., any form o...",[Inc. upholds the highest ethical standards in...,Favoritism or nepotism,0.920672,0.933333,1.000000,0.7500
3,What is the significance of emotional and aest...,Emotional and aesthetic labor play a significa...,[LabourEmotional and aesthetic labor involves ...,Emotional and aesthetic labor in the workplace...,0.969004,0.944444,1.000000,1.0000
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,[see you thrive at Tech Innovators Inc. Welcom...,The purpose of the orientation session at Tech...,1.000000,0.214286,0.000000,0.0000
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., employees can report ...",[and identify areas for improvement.5.3 Report...,Employees can report violations of labor laws ...,0.962563,0.071429,0.250000,1.0000
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.960525,0.625000,1.000000,1.0000
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[IntroductionThis guide provides a step-by-ste...,To extract data from Confluence and create a R...,0.936749,0.500000,0.181818,1.0000
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,[IntroductionTech Innovators Inc. is committed...,Tech Innovators Inc. promotes employee engagem...,0.980843,0.411765,0.800000,0.2500


In [65]:
file_name = "eval_result_test_dataset_it_baseline.csv"
file_path = os.path.join(from_root(),"data-test/eval-result/", file_name)
df.to_csv(file_path, index=False)